# <center> 🦏6.1 - Автоматический стэкинг и блендинг </center>

<div class="alert alert-info">

Как вы уже могли заметить, чем больше моделей вы стэкаете, тем больше у вас разрастается код, а кол-во беспорядка в нем растет по экспоненте. Но есть специальные инструменты, которые позволяют сделать это элегантно и даже более эффективно. Сегодня мы поговорим про sklearn.Pipelines - способ упаковать ваш процесс обучения и инференса от Feature Engineeringа до стэкинга 10 моделей в один пайплайн.

# <center> <img src = '../images/pipelines.jpg' width=750>

## Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import lightgbm
import xgboost
import catboost

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sklearn
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder, MinMaxScaler

sklearn.set_config(display='diagram')

## Считываем данные

In [4]:
data = pd.read_csv('../data/car_train.csv')
data.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel


In [5]:
data['car_id'].nunique()

2337

In [6]:
data.shape

(2337, 10)

### Разделим выборку на валидационную и обучающую

In [7]:
# значения таргета закодируем целыми числами
class_names = np.unique(data['target_class'])
data['target_class'] = data['target_class'].replace(class_names, np.arange(data['target_class'].nunique()))

In [8]:
COLS_BEST = data.drop(columns=['car_id', 'target_reg', 'target_class']).columns

In [9]:
X_train, X_val, y_train, y_val = train_test_split(data.drop(['target_class'], axis=1)[COLS_BEST], 
                                                    data['target_class'],
                                                    test_size=.25,
                                                    stratify=data['target_class'],
                                                    random_state=42)
X_train.shape, X_val.shape

((1752, 7), (585, 7))

## Объявим параметры моделей

### CatBoost

In [10]:
params_cat = {'n_estimators' : 100,
          'learning_rate': .03,
          'depth' : 3,
          'verbose': False,
          # 'use_best_model': True,
          # 'cat_features' : cat_cols,
          'text_features': [],
          # 'train_dir' : '/home/jovyan/work/catboost',
          # 'border_count' : 64,
          # 'l2_leaf_reg' : 1,
          # 'bagging_temperature' : 2,
          # 'rsm' : .1,
          'loss_function': 'MultiClass',
          # 'auto_class_weights' : 'Balanced', #try not balanced
          'random_state': 42,
          'use_best_model': False,
          # 'custom_metric' : ['AUC', 'MAP']
         }

cat_model = catboost.CatBoostClassifier(**params_cat)

### LightGbm

In [11]:
params_lgbm = {'num_leaves': 887,
               'n_estimators': 480,
               'max_depth': 7,
               # 'min_child_samples': 1073,
               'learning_rate': 0.05348257149091985,
               # 'min_data_in_leaf': 2,
               # 'feature_fraction': 0.9529134909800754,
               # 'categorical_feature': cat_cols
              }

In [12]:
lgbm_model = lightgbm.LGBMClassifier(**params_lgbm)

### XGBoost

In [13]:
params_xgb = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    # 'objective': 'reg:linear',
    # 'eval_metric': 'accuracy'
}

In [14]:
xgb_model = xgboost.XGBClassifier(**params_xgb)

## Построим пайплан

### Предобработаем данные

In [15]:
categorical_features = ['model', 'car_type', 'fuel_type']
numerical_features = ['car_rating', 'year_to_start', 'riders', 'year_to_work']

In [16]:
# заменяет пропуски самым частым значением и делает ohe
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))])

In [17]:
# заменяет пропуски средним значением и делает номрализацию
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler())
])

In [18]:
# соединим два предыдущих трансформера в один
preprocessor = ColumnTransformer(transformers=[
    ("numerical", numerical_transformer, numerical_features),
    ("categorical", categorical_transformer, categorical_features)])

In [19]:
preprocessor.transformers[0]

('numerical',
 Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())]),
 ['car_rating', 'year_to_start', 'riders', 'year_to_work'])

### Обучим ансамбль

In [48]:
# список базовых моделей
estimators = [
    ("svm", LinearSVC(verbose=False)),
    ("random_forest", RandomForestClassifier(n_jobs=-1, verbose=False)),
    ("xgboost", xgb_model),
    ("lightgbm", lgbm_model),
    ("catboost", cat_model),
]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False),
    n_jobs=-1,
    verbose=False,
)

stacking_classifier = make_pipeline(preprocessor, meta_model)

In [44]:
stacking_classifier.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['car_rating',
                                                   'year_to_start', 'riders',
                                                   'year_to_work']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['m...
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None, ...)),
                                                ('lightgbm',
                                                 LGBMClassifier(learning_rate=0.05348257149091985,
                                                                max_depth=7,
                                                                n_estimators=480,
                                                                num_leaves=887)),
                                                ('catboost',
                                                 <catboost.core.CatBoostClassifier object at 0x176e86d10>)],
                                    final_estimator=LogisticRegression(random_state=42,
                                                                       verbose=False),
                                    n_jobs=-1, verbose=False))])

In [45]:
for name, model in stacking_classifier.steps[1][1].named_estimators_.items():
    preprocessed = stacking_classifier.steps[0][1].transform(X_val)
    print(name, 'accuracy: ', round(accuracy_score(model.predict(preprocessed), y_val), 4))

svm accuracy:  0.1265
random_forest accuracy:  0.1145
xgboost accuracy:  0.1197
lightgbm accuracy:  0.106
catboost accuracy:  0.1214


In [46]:
print('ensemble score:', round(accuracy_score(stacking_classifier.predict(X_val), y_val), 4))
# как вы можете заметить ансамбль довольно заметно улучшил качество решения

ensemble score: 0.1453


## Выводы

<div class="alert alert-info">

sklearn.Pipelines это очень сильный инструмент, позволяющий упаковать весь процесс обучения модели в один механизм, в который легко добавлять новые модели и который легко применять на инференсе. Кстати, это один из тех инструментов, который часто используется не только на соревнованиях, но и в обычной работе засчет своей элегантности и простоты.